<a href="https://colab.research.google.com/github/alekhyam94/DDL-Project/blob/master/code/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


# Download Training data
Fetch frame level Youtube 8M dataset

In [2]:
!mkdir -p /content/2/frame/train
os.chdir('/content/2/frame/train')
!curl data.yt8m.org/download.py | shard=1,100 partition=2/frame/train mirror=us python

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0   7453      0 --:--:-- --:--:-- --:--:--  7453
Starting fresh download in this directory. Please make sure you have >2TB of free disk space!
>> Downloading http://data.yt8m.org/2/download_plans/frame_train.json 102.7%Succesfully downloaded 2_frame_train_download_plan.json 215275 bytes.
Files remaining 41
Downloading: train0111.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/trainbX.tfrecord 100.0%Succesfully downloaded train0111.tfrecord 275729962 bytes.
Successfully downloaded train0111.tfrecord


Downloading: train2896.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/trainUS.tfrecord 100.0%Succesfully downloaded train2896.tfrecord 275625069 bytes.
Successfully downloaded train2896.tfrecord


Downloading: train3607.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/t

Fetch Frame level validation and training Youtube 8M dataset

In [3]:
!mkdir -p /content/3/frame/test
!mkdir -p /content/3/frame/validate

# Fetch test data
os.chdir('/content/3/frame/test')
!curl data.yt8m.org/download.py | shard=1,100 partition=3/frame/test mirror=us python

# Fetch validation data
os.chdir('/content/3/frame/validate')
!curl data.yt8m.org/download.py | shard=1,100 partition=3/frame/validate mirror=us python

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0   289k      0 --:--:-- --:--:-- --:--:--  289k
Starting fresh download in this directory. Please make sure you have >2TB of free disk space!
>> Downloading http://data.yt8m.org/3/download_plans/frame_test.json 100.7%Succesfully downloaded 3_frame_test_download_plan.json 211431 bytes.
Files remaining 40
Downloading: test0735.tfrecord
>> Downloading http://us.data.yt8m.org/3/frame/test/testl1.tfrecord 100.0%Succesfully downloaded test0735.tfrecord 4103010 bytes.
Successfully downloaded test0735.tfrecord


Downloading: test2246.tfrecord
>> Downloading http://us.data.yt8m.org/3/frame/test/testKo.tfrecord 100.1%Succesfully downloaded test2246.tfrecord 5942646 bytes.
Successfully downloaded test2246.tfrecord


Downloading: test1956.tfrecord
>> Downloading http://us.data.yt8m.org/3/frame/test/testFI.tfreco

# Training
Train LSTM Model on Frame Level features

In [0]:
os.chdir('/content/drive/My Drive/youtube-8m-code/code/youtube-8m/')
!python train.py --frame_features --model=LstmModel --feature_names='rgb' --feature_sizes='1024' --train_data_pattern=/content/2/frame/train/train*.tfrecord --train_dir /content/models/frame/lstm_50 --start_new_model --num_epochs 50 --batch_size 128

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




W1107 19:20:23.933348 140496047077248 module_wrapper.py:139] From train.py:690: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1107 19:20:23.933595 140496047077248 module_wrapper.py:139] From train.py:690: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1107 19:20:23.933734 140496047077248 module_wrapper.py:139] From train.py:691: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:/job:master/task:0: Tensorflow version: 1.15.0.
I1107 19:20:23.933859 140496047077248 train.py:6

# Evaluation

Get evaluation metrics value on Validation dataset

In [0]:
os.chdir('/content/drive/My Drive/youtube-8m-code/code/youtube-8m/')
!python eval.py -eval_data_pattern=/content/3/frame/validate/validate*.tfrecord --train_dir /content/models/frame/lstm_50 --segment_labels --run_once True

# Visualization

In [0]:
os.chdir('/content/models/video/')
%load_ext tensorboard
%tensorboard --logdir random_model

/content/2/frame/train
